# WHY FAKE NEWS IS A PROBLEM?
**Fake news refers to misinformation, disinformation or mal-information which is spread through word of mouth and traditional media and more recently through digital forms of communication such as edited videos, memes, unverified advertisements and social media propagated rumours.Fake news spread through social media has become a serious problem, with the potential of it resulting in mob violence, suicides etc as a result of misinformation circulated on social media.**
![image.png](attachment:image.png)

# BRIEF DESCRIPTION OF DATASET
**This dataset consists of about 40000 articles consisting of fake as well as real news. Our aim is train our model so that it can correctly predict whether a given piece of news is real or fake.The fake and real news data is given in two separate datasets with each dataset consisting around 20000 articles each.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls
%cd /content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset

drive  sample_data
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


In [4]:
%cd fake-and-real-news-dataset/

[Errno 2] No such file or directory: 'fake-and-real-news-dataset/'
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


# LOADING THE NECESSARY LIBRARIES

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# IMPORTING THE DATASET

# DATA VISUALIZATION AND PREPROCESSING

**Import data in tf dataset**

In [6]:
true_dataset = tf.data.experimental.CsvDataset(filenames = ['True.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)
false_dataset = tf.data.experimental.CsvDataset(filenames = ['Fake.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)

In [7]:
for idx, row in enumerate(true_dataset):
    
    if idx == 3:
      print (row)
      break

(<tf.Tensor: shape=(), dtype=string, numpy=b'FBI Russia probe helped by Australian diplomat tip-off: NYT'>, <tf.Tensor: shape=(), dtype=string, numpy=b'WASHINGTON (Reuters) - Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton, the New York Times reported on Saturday. The conversation between Papadopoulos and the diplomat, Alexander Downer, in London was a driving factor behind the FBI\xe2\x80\x99s decision to open a counter-intelligence investigation of Moscow\xe2\x80\x99s contacts with the Trump campaign, the Times reported. Two months after the meeting, Australian officials passed the information that came from Papadopoulos to their American counterparts when leaked Democratic emails began appearing online, according to the newspaper, which cited four current and former U.S. and foreign officials. Besides the information from the Australians, the probe by the Federal Bu

**Join text and title**

In [8]:
def preprocess_join_sentence(s1, s2):
    ret = tf.strings.reduce_join([s1, s2],separator=" ", axis=-1)
    return ret

true_dataset = true_dataset.map(preprocess_join_sentence)
false_dataset = false_dataset.map(preprocess_join_sentence)

In [9]:
for idx, row in enumerate(true_dataset):
  if idx == 3:
    print (row)
    break

tf.Tensor(b'FBI Russia probe helped by Australian diplomat tip-off: NYT WASHINGTON (Reuters) - Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton, the New York Times reported on Saturday. The conversation between Papadopoulos and the diplomat, Alexander Downer, in London was a driving factor behind the FBI\xe2\x80\x99s decision to open a counter-intelligence investigation of Moscow\xe2\x80\x99s contacts with the Trump campaign, the Times reported. Two months after the meeting, Australian officials passed the information that came from Papadopoulos to their American counterparts when leaked Democratic emails began appearing online, according to the newspaper, which cited four current and former U.S. and foreign officials. Besides the information from the Australians, the probe by the Federal Bureau of Investigation was also propelled by intelligence from other friendly gov

### Add labels to dataset

In [10]:
cat_true = tf.data.Dataset.from_tensor_slices(np.ones(len(list(true_dataset)), dtype=int))
cat_false = tf.data.Dataset.from_tensor_slices(np.zeros(len(list(false_dataset)), dtype=int))

In [11]:
true_dataset = tf.data.Dataset.zip((true_dataset, cat_true))
false_dataset = tf.data.Dataset.zip((false_dataset, cat_false))

In [34]:
full_dataset = false_dataset.concatenate(true_dataset)

In [13]:
for idx, row in enumerate(full_dataset):
  if idx == 3:
    print (row)
    break

(<tf.Tensor: shape=(), dtype=string, numpy=b' Trump Is So Obsessed He Even Has Obama\xe2\x80\x99s Name Coded Into His Website (IMAGES) On Christmas day, Donald Trump announced that he would  be back to work  the following day, but he is golfing for the fourth day in a row. The former reality show star blasted former President Barack Obama for playing golf and now Trump is on track to outpace the number of golf games his predecessor played.Updated my tracker of Trump s appearances at Trump properties.71 rounds of golf including today s. At this pace, he ll pass Obama s first-term total by July 24 next year. https://t.co/Fg7VacxRtJ pic.twitter.com/5gEMcjQTbH  Philip Bump (@pbump) December 29, 2017 That makes what a Washington Post reporter discovered on Trump s website really weird, but everything about this administration is bizarre AF. The coding contained a reference to Obama and golf:  Unlike Obama, we are working to fix the problem   and not on the golf course.  However, the coding 

**Formate sentences**

In [35]:
from nltk.corpus import stopwords
nltk.download('stopwords')

def rm_stopwords(text):
  stop = set(stopwords.words('english'))
  nltk.download('punkt')

  stop = list(stop)
  ret = tf.strings.strip(text)

  for i in stop:
    ret = tf.strings.regex_replace(ret, '\\b'+i+'\\b', '')
  # Remove adding spaces

  ret = tf.strings.regex_replace(ret, ' +', ' ')
  return ret

def preprocess_sentence(sentence, label):
  ret = tf.strings.lower(sentence) # to lower

  ret = tf.strings.strip(ret) # create list of words 

  # Denoise text
  ret = tf.strings.regex_replace(ret, r'[^\x00-\x7f]','') # Removing adding chars to decode in UTF-8
  ret = tf.strings.regex_replace(ret, '<[^<]+?>', '') # Removing HTML chars
  ret = tf.strings.regex_replace(ret, '\[[^]]*\]', '') # Removing the square brackets
  ret = tf.strings.regex_replace(ret, 'http\S+', '') # Removing URL's

  ret = tf.strings.regex_replace(ret, '!"#$%&(),-./:;<=>?@\t\n', '') # Keras tokenizer filter
  ret = tf.strings.regex_replace(ret, '[^A-Za-z0-9]+', ' ') # remove ponctuation

  ret = rm_stopwords(ret) # Removing stopwords

  return ret, label

full_dataset = full_dataset.map(preprocess_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
for idx, row in enumerate(full_dataset):
  if idx == 3:
    print(row)
    break
    
    

(<tf.Tensor: shape=(), dtype=string, numpy=b'trump obsessed even obamas name coded website images christmas day donald trump announced would back work following day golfing fourth day row former reality show star blasted former president barack obama playing golf trump track outpace number golf games predecessor played updated tracker trump appearances trump properties 71 rounds golf including today pace pass obama first term total july 24 next year pic twitter com 5gemcjqtbh philip bump pbump december 29 2017 makes washington post reporter discovered trump website really weird everything administration bizarre af coding contained reference obama golf unlike obama working fix problem golf course however coding done correctly website donald trump spent several days row golf course coded serve following message event internal server error pic twitter com wiqsqnnzw0 christopher ingraham cingraham december 28 2017that snippet code appears pages footer says paid rnc pic twitter com oazdt126

In [16]:
!pip install -q tensorflow-text


     |████████████████████████████████| 3.4MB 9.2MB/s 


### Glove

**Count max occurences in dataset**

In [40]:
from collections import Counter

def freq_words(inputs, max_size=10500): # input dataset, returns list
  count = Counter()

  numpy_sentences = []
  for sentences, _ in full_dataset.take(-1):  # take all elements in dataset
      numpy_sentences.extend(tf.strings.split(sentences).numpy())
  list_words = numpy_sentences

  for word in list_words:
    # print(word)
    count[word] += 1
  result = [word for word, num in count.most_common(max_size)]
  return result

max_freq_words = freq_words(full_dataset)

In [41]:
max_freq_words = [w.decode("utf-8") for w in max_freq_words]

In [42]:
max_freq_words[:10]

['trump',
 'said',
 'u',
 'president',
 'would',
 'people',
 'one',
 'state',
 'new',
 'also']

**Find this occurences in the pretrained glove file**

In [43]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Build vocabulary and relevent vector
words = []
# Adding 2 empty vector of dimension 100 which will be used in padding and unknown token(word)
# Added extra 2 rows because when we will use inside TextVectorization add blank i.e. '' and ['UNK'] inside vocabulary

counter = 0

vectors = np.zeros((10000, 100))

with open(f"../glove-twitter/glove.twitter.27B.100d.txt") as f:
    lines = f.readlines()

    for idx, line in enumerate(lines):
      if counter >= 10000-2:
        break

      split_line = line.split()
      
      if split_line[0] in max_freq_words:
        words.append(split_line[0])
        vectors[counter+2] = split_line[1:]
        counter+=1

print(f"Number of words are {len(words)}")
print(f"Shape of vector is {vectors.shape}")

Number of words are 9998
Shape of vector is (10000, 100)


In [44]:
vectors[:3]

array([[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0. 

In [45]:
words[1120:1130]

['dire',
 'available',
 'code',
 'ryan',
 'shake',
 'gas',
 'gt',
 'handle',
 'putting',
 'mario']

In [46]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# TextVectorization hyper parameter
max_vocab = 10000
max_len = 300
vectorize_layer_glove = TextVectorization(max_tokens=max_vocab,
                                                    standardize="lower_and_strip_punctuation",
                                                    split="whitespace",
                                                    output_mode="int",
                                                    output_sequence_length=max_len,
                                                    vocabulary=words)


# Vocabulary set into layer
vectorize_layer_glove.set_vocabulary(words)
print(f"Top 10 words {vectorize_layer_glove.get_vocabulary()[:10]}")
print(f"Some words {vectorize_layer_glove.get_vocabulary()[-1010:-1000]} \n")

Top 10 words ['', '[UNK]', 'rt', 'de', 'n', 'la', 'e', 'u', 'en', 'el']
Some words ['fec', 'migrant', 'provocation', 'identifies', 'prolonged', 'manus', 'autonomy', 'clad', 'classification', 'sectors'] 



In [47]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer_glove(text), label

train_data_tokenized = full_dataset.map(int_vectorize_text)

In [49]:
model_glove = tf.keras.models.Sequential()
model_glove.add(tf.keras.layers.Input(shape=(1,), dtype=tf.string))
model_glove.add(vectorize_layer_glove)
model_glove.predict(["expect terror fbi steps use isis stings 21st century wire says real terrorist threat thing ..."])

array([[ 944, 2670, 3911, 2360,  296, 6331,    1,    1, 3467, 4046,  333,
         105, 4255, 3693,  155,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [50]:
for idx, row in enumerate(train_data_tokenized):
  if idx == 1:
    print(row)
    break

(<tf.Tensor: shape=(1, 300), dtype=int64, numpy=
array([[ 618, 5639, 5604, 9205,  535, 2885, 9481, 4992,  209, 3438, 4647,
        5234, 5306, 7651,   66,  131,   28, 7473,   10,  216,   96, 3953,
        7436, 6671, 9081, 2248, 4992, 9247, 3771, 1981, 3911,  754, 1998,
        5604,  725, 6671,  535, 4992, 2119, 5933, 7966,   27,  937,  307,
        2249, 5604, 2176, 7482, 1378, 9623,  618, 1312,  810, 3930, 2532,
        2885, 4663, 2131, 5015, 3749,  240, 9623,    1,  237, 5604, 4911,
        5845,   60, 5607, 1707, 1142, 9304,    1, 1242, 1312,  810,  812,
         294,  195, 6061,   27,  937, 1094, 5604, 1075, 8458, 9047,    9,
        1941, 2331,  223,  691, 2806, 1456, 2249, 5737,  444, 4794,  137,
        2945, 5604,  103, 5604, 1382, 5593,  812,  237,  203,    1, 9623,
        3930, 3383, 7773, 4088,    1, 2885, 4078,  672,  227,  981, 3258,
        5578, 4131, 6767, 5015, 3749,  678,   60, 1091, 9623,  152,   71,
           1, 1312, 4035, 3383, 4088,    1, 8271, 1150, 2021,  

In [51]:
len(vectors[0])

100

### Prepare data to train

In [52]:
# Model params
batch_size = 256
epochs = 10
embed_size = 100
maxlen = 300
max_features = 10000

**Model**

In [53]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, weights=[vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True))
model.add(LSTM(units=64))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)


In [54]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 300, 128)          117248    
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________


### TODO : add validation dataset and shuffle data

In [55]:
full_dataset = train_data_tokenized.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

In [56]:
len(list(full_dataset))

44898

In [57]:
DATASET_SIZE = len(list(full_dataset))

train_size = int(0.8 * DATASET_SIZE)
test_size = int(0.2 * DATASET_SIZE)

full_dataset = full_dataset.shuffle(buffer_size=DATASET_SIZE, seed=42)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)

In [58]:
for idx, row in enumerate(train_dataset):
  if idx == 1:
    print(row)
    break

(<tf.Tensor: shape=(1, 300), dtype=int64, numpy=
array([[5604,  326, 3903, 6114, 2248, 4992,   10, 7512,    1, 3962, 5604,
         693, 6979,   60,   56,   25,   33, 5604,  158,  796,  474, 2283,
         240, 1269, 6360, 5360,  111,  740, 6289,  398, 3783, 2106, 2914,
        5067,  796, 3783, 4186,  339,  194,   56, 1944, 5604, 2561,   63,
        2718, 2175,   63,  398, 5604, 1747, 6114, 2248, 4992, 9027, 1981,
        4992, 2344,  500, 8268, 1911, 8463, 5604, 2321, 1652, 5683, 8519,
           1, 3649,  532,  564,  303,  125, 5604, 8884, 2217,    1,  691,
        8433, 4647, 8574, 9481, 6899,  387,  724,  154, 2235,  334, 3962,
         276, 5604,    1, 1279,    1,    1,  520,  244,  564, 1714,  296,
        1156, 2303,  532,  564,    1,  680,   30,   22,  227, 3962,  276,
        5604,    1, 1279,    1,    1, 6060, 4331, 3078,   66,  785, 3887,
         772, 1925,  124, 7146,   85,  772, 5889,   85, 3962,  276, 5604,
           1, 1279,    1,    1, 7903, 5683, 3911, 2327,   78,  

In [59]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11152789302019538413
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8051534296310860513
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [ ]:
# Train for 2 epochs
model.fit(train_dataset, batch_size = batch_size, validation_data=test_dataset, epochs=2, steps_per_epoch=20000)